In [27]:
from glob import glob
from os import path
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_objval

In [28]:
list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
print(list_city_dt)
print(len(list_city_dt))

['frankfurt_db=4.00', 'berlin_db=0.50', 'paris_db=1.00', 'lyon_db=2.00', 'paris_db=4.00', 'berlin_db=1.00', 'lyon_db=1.00', 'lyon_db=4.00', 'frankfurt_db=0.50', 'berlin_db=2.00', 'berlin_db=4.00', 'frankfurt_db=1.00', 'frankfurt_db=2.00', 'paris_db=0.50', 'paris_db=2.00', 'lyon_db=0.50']
16


In [29]:
def dict_append_workforce(dict_, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, workforce_size, objvalprev, objval, objvalnext):
    dict_['instance_file_base'].append(base_file)
    dict_['city'].append(city)
    dict_['demand_baseline'].append(demand_baseline)
    dict_['model'].append(model)
    dict_['max_n_shifts'].append(max_n_shifts)
    dict_['outsourcing_cost_multiplier'].append(OC)
    dict_['regional_multiplier'].append(RM)
    dict_['global_multiplier'].append(GM)
    dict_['workforce_size_region0'].append(workforce_size)
    dict_['objective_value_prev'].append(objvalprev)
    dict_['objective_value'].append(objval)
    dict_['objective_value_next'].append(objvalnext)
    return dict_

In [30]:
import json
base_file = 'berlin_db=1.00'
weekday_ = f'{base_file}_dt=doublepeak.json'
instance_file_weekday = f"../instances/{weekday_}"

# Specify the file path
file_path = '/path/to/your/file.json'

# Open the JSON file
with open(instance_file_weekday, 'r') as file:
    data = json.load(file)

region_data = data['geography']['city']['regions']
region_ids = [region['id'] for region in region_data]

# build workforce dictionary
workforce_dict = {}
for region in range(max(region_ids)+1):
    workforce_dict[region] = 1


In [31]:
workforce_dict[0] = 5
workforce_dict

{0: 5, 1: 1, 2: 1, 3: 1}

In [32]:
list_results = []

MAX_TRIES = 35

RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

# for base_file in list_city_dt:
for base_file in ['berlin_db=1.00']:

    #create inputs to run a trial workforce size
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    instance_file_weekday = f"../instances/{weekday_}"
    instance_file_weekend = f"../instances/{weekend_}"

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"

    #create dictionary for evaluating optimal workforce size
    workforce_out = f"../workforce_size/{base_file}.json"
    dict_out = {
        'instance_file_base':[],
        'city':[],
        'demand_baseline':[],
        'model':[],
        'max_n_shifts':[],
        'outsourcing_cost_multiplier':[],
        'regional_multiplier':[],
        'global_multiplier':[],
        'workforce_size_region0':[],
        'objective_value_prev':[],
        'objective_value':[],
        'objective_value_next':[]
    }

    city_pattern = r'(\w+)_db'
    db_pattern = r'db=(\d+\.\d+)'

    city_match = re.search(city_pattern, base_file)
    db_match = re.search(db_pattern, base_file)

    city = city_match.group(1) if city_match else None
    demand_baseline = float(db_match.group(1)) if db_match else None

    ### Build starting workforce dictionary
    with open(instance_file_weekday, 'r') as file:
        data = json.load(file)

    # Extract region IDs
    region_data = data['geography']['city']['regions']
    region_ids = [region['id'] for region in region_data]

    # build workforce dictionary
    workforce_dict = {}
    for region in range(max(region_ids)+1):
        workforce_dict[region] = 1

    # for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
    for OC in [1.2]:
        # for model in ['fixed', 'flex', 'partflex']:
        for model in ['fixed']:
            if model == 'partflex':
                # for max_n_shifts in range(2,5):
                for max_n_shifts in [4]:
                    objvalprev = 1e8
                    objval = 1e7
                    for trial_size in range(1, MAX_TRIES):
                        workforce_dict[0] = trial_size
                        if (objval < objvalprev)&(np.isnan(objval) == False):
                            objvalprev = objval
                            dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts)
                            objval = dict_results['objective_value'][0]
                            print("trial_size",trial_size)
                            df_ = pd.DataFrame(dict_results)
                            df_['workforce_size_trial'] = trial_size
                            list_results.append(df_)
                            print("trial_size",trial_size)
                        else:
                            #keep track of optimal
                            dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size-1, np.nan, objvalprev, objval)
                            break
                        if trial_size == int(MAX_TRIES-1):
                            #keep track of optimal
                            dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, max_n_shifts, trial_size, objvalprev, objval, np.nan)
            else:
                objvalprev = 1e8
                objval = 1e7
                for trial_size in range(1, MAX_TRIES):
                    workforce_dict[0] = trial_size
                    if (objval < objvalprev)&(np.isnan(objval) == False):
                        objvalprev = objval
                        dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff)
                        objval = dict_results['objective_value'][0]
                        print("trial_size",trial_size)
                        df_ = pd.DataFrame(dict_results)
                        df_['workforce_size_trial'] = trial_size
                        list_results.append(df_)
                    else: 
                        #keep track of optimal (trial size - 1)
                        dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, np.nan, trial_size-1, np.nan, objvalprev, objval)
                        break
                    if trial_size == int(MAX_TRIES-1):
                        #keep track of optimal
                        dict_out = dict_append_workforce(dict_out, base_file, city, demand_baseline, OC, RM, GM, model, np.nan, trial_size, objvalprev, objval, np.nan)
    #export optimal workforce size values
    with open(workforce_out, 'w') as f:
        json.dump(dict_out, f, indent=2)


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-12
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 99724 rows, 102488 columns and 157416 nonzeros
Model fingerprint: 0xc98a953b
Variable types: 99120 continuous, 3368 integer (3368 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 99724 rows and 102488 columns
Presolve time: 0.08s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.12 seconds (0.11 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 7587.05 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.587053333328e+03, best bound 7.587053333328e+03, gap 0.0000%
trial_size 1
Gurobi Optimizer version 10.0.3

FileNotFoundError: [Errno 2] No such file or directory: '../workforce_size/berlin_db=1.00.json'

In [ ]:
#export a concat of all the trial information

df_trials = pd.concat(list_results, ignore_index = True)
df_trials.to_csv(r'../workforce_size/all_trials.xlsx', index = False)
